In [ ]:
import cv2
import math

from ultralytics import YOLO

In [ ]:
import pytesseract
import csv
from pytesseract import Output

In [ ]:
import pytesseract
from pytesseract import Output

pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

# Lenguajes disponibles
print(pytesseract.get_languages(config=''))

model1 = YOLO('C:/Users/danif/Downloads/best_vc.pt')

model2 = YOLO('yolo11n.pt')

# Liste di classi per ciascun modello
classNames_model1 = ["matricula"]
classNames_model2 = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

# Cattura video
vid = cv2.VideoCapture('C:/Users/danif/Downloads/C0142.MP4')

plates_images = []

while True:
    ret, img = vid.read()
    if not ret:
        break

    # Rilevamento con il primo modello
    results1 = model1(img, stream=True)
    for r in results1:
        for box in r.boxes:
            cls = int(box.cls[0])
            if cls < len(classNames_model1):
                class_name = classNames_model1[cls]
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = round(float(box.conf[0]) * 100, 2)

                #ritaglio l'immagine della targa
                plate_image = img[y1:y2, x1:x2]

                # Aggiungi l'immagine della targa alla lista
                plates_images.append(plate_image)

                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(img, f'{class_name} {confidence}%', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    # Rilevamento con il secondo modello
    results2 = model2(img, stream=True)
    for r in results2:
        for box in r.boxes:
            cls = int(box.cls[0])
            if cls < len(classNames_model2):
                class_name = classNames_model2[cls]
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = round(float(box.conf[0]) * 100, 2)
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, f'{class_name} {confidence}%', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Mostra l'immagine con le rilevazioni di entrambi i modelli
    cv2.imshow("Detections with Two Models", img)
    
    # Interruzione con ESC
    if cv2.waitKey(1) == 27:
        break

# Libera le risorse
vid.release()
cv2.destroyAllWindows()

Funzione per controllare la correttezza della targa

In [ ]:
import re
plate_pattern = re.compile(r'^\d{4}\s[A-Z]{3}$')
def validate(plate):  

    if plate_pattern.match(plate):
        
        return plate
    else:
        
        plate_refactored = re.sub(r'^\s*.*?(\d{4}\s[A-Z]{3}).*$', r'\1', plate)
        return plate_refactored


In [ ]:
import numpy as np
extracted_texts = []
matriculas = []

for plate_img in plates_images:

    if plate_img is not None:
    #Convierte a RGB antes de procesar
        img_rgb = cv2.cvtColor(plate_img, cv2.COLOR_BGR2RGB)
       
       
        # Preprocessing dell'immagine della targa per migliorare l'OCR 
        
        gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
        #filtered_img = cv2.GaussianBlur(gray, (5, 5), 0)
        clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
        contrast_img = clahe.apply(gray)
        resized_img = cv2.resize(contrast_img, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
        kernel = np.array([[0, -2, 0], [-2, 9, -2], [0, -2, 0]])
        sharpened_img = cv2.filter2D(resized_img, -1, kernel)

        #Texto y localización en imagen de cada palabra
        text = pytesseract.image_to_string(sharpened_img, config='--psm 8', output_type=Output.STRING)

        text = validate(text)

        if plate_pattern.match(text):
            matriculas.append(text)
            print("\nMATRÍCULA DETECTADA:", text)

        cv2.imshow('img', sharpened_img)
        cv2.waitKey(0)

   

else:
    print('Error de imagen')




In [ ]:
dup = set()  # Insieme per tracciare elementi già visti

for item in matriculas:
    if item not in dup:
        print("\nMatricula: ", item)
        dup.add(item)

In [ ]:
import cv2
import re
import numpy as np
import pytesseract
from pytesseract import Output
from ultralytics import YOLO
import csv

# Configuración de Tesseract para OCR
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

# Carga de modelos YOLO
model1 = YOLO("C:/Users/danif/Downloads/best_vc.pt")  # Modelo entrenado para matrículas
model2 = YOLO('yolo11n.pt')  # Modelo para objetos comunes

# Listas de clases para cada modelo
classNames_model1 = ["matricula"]
classNames_model2 = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

# Expresión regular para detectar matrículas en el formato correcto
plate_pattern = re.compile(r'^\d{4}\s[A-Z]{3}$')

# Ruta del video de entrada y salida
input_video_path = "C:/Users/danif/Downloads/C0142.MP4"
output_video_path = 'output_with_detections.mp4'

# Configuración de captura de video y salida
vid = cv2.VideoCapture(input_video_path)
fps = int(vid.get(cv2.CAP_PROP_FPS))
frame_width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Archivo CSV para registrar los datos
csv_file_path = 'detections.csv'
csv_headers = ['frame', 'object_class', 'confidence', 'x1', 'y1', 'x2', 'y2', 'plate_text']

# Inicializar listas de detecciones y OCR
plates_images = []
extracted_texts = []
matriculas = []
detected_data = []

# Contadores acumulativos de personas y coches en todo el video
person_count_total = 0
car_count_total = 0

# Iniciar la escritura en CSV
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(csv_headers)

    frame_idx = 0
    while True:
        ret, img = vid.read()
        if not ret:
            break

        # Contadores de personas y coches por cada fotograma
        person_count = 0
        car_count = 0

        # Detección de matrículas
        results1 = model1(img, stream=True)
        for r in results1:
            for box in r.boxes:
                cls = int(box.cls[0])
                if cls < len(classNames_model1):
                    class_name = classNames_model1[cls]
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    confidence = round(float(box.conf[0]) * 100, 2)

                    # Recortar la imagen de la matrícula
                    plate_image = img[y1:y2, x1:x2]
                    plates_images.append(plate_image)

                    # Realizar OCR en la imagen de la matrícula
                    gray = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)
                    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
                    contrast_img = clahe.apply(gray)
                    resized_img = cv2.resize(contrast_img, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
                    kernel = np.array([[0, -2, 0], [-2, 9, -2], [0, -2, 0]])
                    sharpened_img = cv2.filter2D(resized_img, -1, kernel)
                    text = pytesseract.image_to_string(sharpened_img, config='--psm 8', output_type=Output.STRING)
                    text = re.sub(r'^\s*.*?(\d{4}\s[A-Z]{3}).*$', r'\1', text) if not plate_pattern.match(text) else text

                    # Registro en CSV
                    writer.writerow([frame_idx, class_name, confidence, x1, y1, x2, y2, text])

                    # Visualización
                    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(img, f'{class_name} {confidence}%', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        # Detección de otros objetos
        results2 = model2(img, stream=True)
        for r in results2:
            for box in r.boxes:
                cls = int(box.cls[0])
                if cls < len(classNames_model2):
                    class_name = classNames_model2[cls]
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    confidence = round(float(box.conf[0]) * 100, 2)

                    # Incrementar contadores de personas y coches
                    if class_name == "person":
                        person_count += 1
                        person_count_total += 1
                    elif class_name == "car":
                        car_count += 1
                        car_count_total += 1

                    # Dibujar las detecciones
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(img, f'{class_name} {confidence}%', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Mostrar conteos de personas y coches en la esquina superior izquierda
        cv2.putText(img, f'Personas (Frame): {person_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(img, f'Coches (Frame): {car_count}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Mostrar conteos acumulativos en la esquina superior derecha
        cv2.putText(img, f'Personas (Total): {person_count_total}', (frame_width - 300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.putText(img, f'Coches (Total): {car_count_total}', (frame_width - 300, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        # Mostrar el fotograma procesado en tiempo real
        cv2.imshow("Detections with Two Models", img)

        # Escribir el fotograma procesado en el archivo de video de salida
        out.write(img)
        frame_idx += 1

        # Permite salir del bucle pulsando la tecla 'Esc'
        if cv2.waitKey(1) == 27:
            break

# Liberar recursos
vid.release()
out.release()
cv2.destroyAllWindows()
